In [2]:
%load_ext autoreload
%autoreload 2

import os 
import matplotlib
from ipywidgets import interactive, widgets
import plotly.graph_objects as go
from src.utils import *
from src.session import * 
from src.signal_processing import *
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']= 100

%matplotlib qt5
plt.ion()

### Load data

In [3]:
dir = 'path/to/data/'
channel_layout_dir = 'channel_layout.json'

session = LFPSession(dir, channel_layout_dir=channel_layout_dir)

#### Select start and end time

In [59]:
start_time, end_time = 200, 500 # in seconds
# int(start_time * session.fs),int(end_time * session.fs)
data = session.load_data(int(start_time * session.fs),int(end_time * session.fs))

#### Select channels

In [60]:
chs = session.get_channels('left', min_depth = 14, max_depth=15) # 0-indexed
data.set_channels(chs)

#### Plot waveforms (probably better to preprocess first though)

In [61]:
plot_waveforms(data,session)

### Preprocess

In [67]:
ds_factor = 50
raw_samples = data.samples
preprocessed_samples = preprocess(raw_samples, fs = session.fs, ds=ds_factor, low_pass_cutoff=250, filter_order=5)
fs_new = session.fs/ds_factor

In [11]:
plot_waveforms(data,session,samples=preprocessed_samples)

### Select a channel for further analysis

In [74]:
selected_samples = preprocessed_samples[:,0]

### Analysis

#### Frequency spectra

In [12]:
bins, fft = get_fft(selected_samples, fs_new, norm=True)
plot_psd(bins, fft,num_samples=selected_samples.shape[0], xleft=0, xright=100)

#### Spectrogram

In [76]:
f, t, Sxx = signal.spectrogram(selected_samples, fs_new)
plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

### Get theta / delta bands

In [77]:
theta = apply_bandpass_filter(selected_samples,lowcut= 5,highcut= 10, fs=fs_new)
theta_as, theta_amp, theta_freq = get_analytic_signal(theta, fs_new)
plot_hilbert(theta_as, theta_amp, theta_freq, fs_new)

/Users/patrickhajali/Library/Caches/pypoetry/virtualenvs/beatrice-brain-n1O5vrzQ-py3.9/lib/python3.9/site-packages/matplotlib/cbook.py:1699: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/Users/patrickhajali/Library/Caches/pypoetry/virtualenvs/beatrice-brain-n1O5vrzQ-py3.9/lib/python3.9/site-packages/matplotlib/cbook.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [78]:
delta = apply_bandpass_filter(selected_samples,lowcut= 0.5,highcut= 4, fs=fs_new)
delta_as, delta_amp, delta_freq = get_analytic_signal(delta, fs_new)
plot_hilbert(delta_as, delta_amp, delta_freq, fs_new, bottom=0.5, top = 6);

: 

#### Plot theta/delta waveforms

In [15]:
plot_waveforms(data,session,samples=preprocessed_samples, other_data= [theta, delta], other_data_labels=['Theta', 'Delta'])

### Ratio of theta/delta

In [72]:
ratio = theta_amp / delta_amp
ratio_threshold = 2

theta_cycles = np.where(ratio > ratio_threshold, 1, 0)

window_size = int(1 * fs_new)

# convolve to identify potential regions
convolved = np.convolve(theta_cycles, np.ones(window_size)/window_size, mode='same')

theta_cycles_entire = np.zeros_like(theta_cycles)
for i in range(len(convolved)):
    # if ratio is met at a certain point, mark the entire window as a theta cycle
    if convolved[i] >= 1: 
        start_index = max(i - window_size // 2, 0)
        end_index = min(i + window_size // 2, len(theta_cycles))
        theta_cycles_entire[start_index:end_index] = 1

np.sum(theta_cycles_entire)/len(theta_cycles_entire)

0.02674